# Day 2 - Exercise 3

Load required libraries, function and built-ins

In [1]:
import pstats
import matmult_functionalized as mf
import matmult_functionalized_opt as mfo
import euler72
%load_ext line_profiler
%load_ext memory_profiler

## 3a)

First we profile the code using the Python inbuilt ```cProfile```.

In [2]:
!python -m cProfile -o matmult_cProfile.txt matmult.py >>/dev/null
p = pstats.Stats('matmult_cProfile.txt')
p.sort_stats('cumulative').print_stats(10)

Mon Mar 30 08:55:18 2020    matmult_cProfile.txt

         726254 function calls (726211 primitive calls) in 5.854 seconds

   Ordered by: cumulative time
   List reduced from 120 to 10 due to restriction <10>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
      4/1    0.000    0.000    5.854    5.854 {built-in method builtins.exec}
        1    5.603    5.603    5.854    5.854 matmult.py:2(<module>)
   125250    0.034    0.000    0.205    0.000 /home/paul/miniconda3/lib/python3.7/random.py:218(randint)
   125250    0.075    0.000    0.171    0.000 /home/paul/miniconda3/lib/python3.7/random.py:174(randrange)
      250    0.016    0.000    0.123    0.000 matmult.py:11(<listcomp>)
      250    0.014    0.000    0.112    0.000 matmult.py:16(<listcomp>)
   125250    0.067    0.000    0.096    0.000 /home/paul/miniconda3/lib/python3.7/random.py:224(_randbelow)
   158635    0.020    0.000    0.020    0.000 {method 'getrandbits' of '_random.Random' objects}
   125250

Since the code is not functionalized, interpreting the profiling output is not that straightforward. We can see from above's output that it seems that the most expensive part of the program starts on line 2, which imports the package ```random```. Reading further, we see, however, that the functions ```randint()``` and ```randrange()``` are only responsible for a minor part of the execuation time. In order to find the real bottleneck, we functionalize the code. The result can be found in the file ```matmult_functionalized.py```.

In [3]:
pff = %prun -r mf.main(print_result=False)
pff.sort_stats('cumulative').print_stats(10)

          724305 function calls in 3.757 seconds

   Ordered by: cumulative time
   List reduced from 16 to 10 due to restriction <10>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    3.757    3.757 {built-in method builtins.exec}
        1    0.001    0.001    3.757    3.757 <string>:1(<module>)
        1    0.000    0.000    3.756    3.756 matmult_functionalized.py:42(main)
        1    3.501    3.501    3.505    3.505 matmult_functionalized.py:7(matmult)
   125250    0.035    0.000    0.213    0.000 random.py:218(randint)
   125250    0.077    0.000    0.178    0.000 random.py:174(randrange)
        1    0.001    0.001    0.128    0.128 matmult_functionalized.py:33(get_Y)
      250    0.018    0.000    0.127    0.001 matmult_functionalized.py:38(<listcomp>)
        1    0.001    0.001    0.124    0.124 matmult_functionalized.py:24(get_X)
      250    0.019    0.000    0.123    0.000 matmult_functionalized.py:29(<listcomp>)




We can see now that the majority of the time is spent inside ```matmult()```. Thus it seems fruitful to optimize this function for speed. In order to verify this, we move on to the ```line_profiler``` results.

In [4]:
lprof_obj = %lprun -r -f mf.matmult -f mf.get_X -f mf.get_Y -f mf.main mf.main(print_result=False)
lprof_obj.print_stats()

Timer unit: 1e-06 s

Total time: 16.3774 s
File: /home/paul/git/ASPP_course/day2_exercises/exc3_profiling/matmult_functionalized.py
Function: matmult at line 7

Line #      Hits         Time  Per Hit   % Time  Line Contents
     7                                           def matmult(X,Y):
     8                                               
     9                                               # result is Nx(N+1)
    10         1          0.0      0.0      0.0      result = []
    11       251         95.0      0.4      0.0      for i in range(len(X)):
    12       250       1375.0      5.5      0.0          result.append([0] * (len(Y[0])))
    13                                               
    14                                               # iterate through rows of X
    15       251        108.0      0.4      0.0      for i in range(len(X)):
    16                                                   # iterate through columns of Y
    17     63000      21561.0      0.3      0.1   

From above's ```line_profiler``` output, we see that the majority of the time (~98%) is spent inside the function ```matmul()```. Inside the function ```matmul()```, most of the time is spent by performing the innermost loop over the result array and computing its values. Therefore lines 19 and 20 would be the point where one would start optimizing the code for speed.

Next we investigate the memory characteristics of our code using the ```memory_profiler```.

In [5]:
%mprun -T matmult_functionalized.py.mprof -f mf.main mf.main(print_result=False)
print(open('matmult_functionalized.py.mprof', 'r').read())



*** Profile printout saved to text file matmult_functionalized.py.mprof. 
Filename: /home/paul/git/ASPP_course/day2_exercises/exc3_profiling/matmult_functionalized.py

Line #    Mem usage    Increment   Line Contents
    42     50.7 MiB     50.7 MiB   def main(print_result=True):
    43                                 """Main function binding together the other functions.
    44                                 """
    45                             #    random.seed(314156345)
    46     50.7 MiB      0.0 MiB       N = 250
    47     50.9 MiB      0.2 MiB       X = get_X(N)
    48     51.4 MiB      0.5 MiB       Y = get_Y(N)
    49     53.8 MiB      2.4 MiB       result = matmult(X,Y)
    50     53.8 MiB      0.0 MiB       if print_result:
    51                                     for r in result:
    52                                         print(r)
    53                                 
    54     53.8 MiB      0.0 MiB       return 0


Above's output is somewhat confusing. We can see that most memory is created when the main function is entered, i.e., when the program starts. The overall memory consumption is, howerver, low and the increment in the program is just a few MB where arrays are populated. Optimizing for memory use seems thus not very necessary.

## 3b)

### Using ```phi()```

In [6]:
ps = %prun -r -q euler72.main(fast=False)
ps.sort_stats('cumulative').print_stats(8)

30397485
          151506 function calls in 10.398 seconds

   Ordered by: cumulative time
   List reduced from 24 to 8 due to restriction <8>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000   10.398   10.398 {built-in method builtins.exec}
        1    0.000    0.000   10.398   10.398 <string>:1(<module>)
        1    0.004    0.004   10.398   10.398 euler72.py:62(main)
     9999   10.347    0.001   10.393    0.001 euler72.py:36(phi)
     9999    0.036    0.000    0.046    0.000 euler72.py:22(factorize)
    99314    0.008    0.000    0.008    0.000 {built-in method math.sqrt}
    32153    0.003    0.000    0.003    0.000 {method 'append' of 'list' objects}
        1    0.001    0.001    0.001    0.001 euler72.py:5(gen_primes)




We see that most of the time is spent inside the function ```phi()```. Thus speed optimization is most effective there. The program comes with an alternative function ```fast_phi()```, which is profiled further below. In order to see, which line takes longest to execute, we look at the output of ```line_profiler```:

In [7]:
lprof_obj = %lprun -r -f euler72.main -f euler72.phi -f euler72.factorize euler72.main(fast=False)
lprof_obj.print_stats()

30397485
Timer unit: 1e-06 s

Total time: 0.220518 s
File: /home/paul/git/ASPP_course/day2_exercises/exc3_profiling/euler72.py
Function: factorize at line 22

Line #      Hits         Time  Per Hit   % Time  Line Contents
    22                                           def factorize(n, primes):
    23      9999       6397.0      0.6      2.9      factors = []
    24      9999       4377.0      0.4      2.0      init_n = n
    25     96347      35473.0      0.4     16.1      for p in primes:
    26    118736      63728.0      0.5     28.9          while(n%p == 0):
    27     22389      13012.0      0.6      5.9              n = n/p
    28     22389      21281.0      1.0      9.7              factors.append(p)
    29     96347      57053.0      0.6     25.9          if(p > sqrt(n)):
    30      9999       3992.0      0.4      1.8              break
    31      9999       5017.0      0.5      2.3      if(n > 1):
    32      9596       6531.0      0.7      3.0          factors.append(n)
 

We see now that most of the time is spent executing lines 41-48 in function ```phi()```. These lines include two loops and logic operations, which should be the point to look out for speed optimizations. Finally we look at the memory consumption of the program using ```memory_profiler```.

In [8]:
%mprun -T euler72_slow.py.mprof -f euler72.main euler72.main(fast=False)
print(open('euler72_slow.py.mprof', 'r').read())

30397485


*** Profile printout saved to text file euler72_slow.py.mprof. 
Filename: /home/paul/git/ASPP_course/day2_exercises/exc3_profiling/euler72.py

Line #    Mem usage    Increment   Line Contents
    62     52.2 MiB     52.2 MiB   def main(fast=True):
    63     52.2 MiB      0.0 MiB       primes = gen_primes(1000)
    64     52.2 MiB      0.0 MiB       m = 10000
    65                                 #m = 8
    66     52.2 MiB      0.0 MiB       fraq = 0
    67     52.2 MiB      0.0 MiB       if fast:
    68                                     for i in range(2,m+1):
    69                                         fraq += fast_phi(i,primes)
    70                                 else:
    71     52.2 MiB      0.0 MiB           for i in range(2,m+1):
    72     52.2 MiB      0.0 MiB               fraq += phi(i,primes)
    73                             
    74     52.2 MiB      0.0 MiB       print(fraq)
    75     52.2 MiB      0.0 MiB       return 0


### Using ```fast_phi()```

In [9]:
pff = %prun -r -q euler72.main()
pff.sort_stats('cumulative').print_stats()

30397485.0
          161505 function calls in 0.055 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.055    0.055 {built-in method builtins.exec}
        1    0.000    0.000    0.055    0.055 <string>:1(<module>)
        1    0.002    0.002    0.055    0.055 euler72.py:62(main)
     9999    0.011    0.000    0.051    0.000 euler72.py:51(fast_phi)
     9999    0.031    0.000    0.040    0.000 euler72.py:22(factorize)
    99314    0.007    0.000    0.007    0.000 {built-in method math.sqrt}
    32153    0.002    0.000    0.002    0.000 {method 'append' of 'list' objects}
        1    0.001    0.001    0.001    0.001 euler72.py:5(gen_primes)
    10000    0.001    0.000    0.001    0.000 {built-in method builtins.len}
        1    0.000    0.000    0.000    0.000 {built-in method builtins.print}
        2    0.000    0.000    0.000    0.000 iostream.py:384(write)
        3    0.000    0.000    

We see that the majority of the time is spent in function ```factorize()``` starting in line 23, which is called by the function ```fast_pi()```, which in turn is called by ```main()```. Thus speed optimization seems to be most fruitful inside the factorize function. In comparison with 

In [10]:
lprof_obj = %lprun -r -f euler72.main -f euler72.fast_phi -f euler72.factorize euler72.main()
lprof_obj.print_stats()

30397485.0
Timer unit: 1e-06 s

Total time: 0.167634 s
File: /home/paul/git/ASPP_course/day2_exercises/exc3_profiling/euler72.py
Function: factorize at line 22

Line #      Hits         Time  Per Hit   % Time  Line Contents
    22                                           def factorize(n, primes):
    23      9999       3381.0      0.3      2.0      factors = []
    24      9999       3313.0      0.3      2.0      init_n = n
    25     96347      31738.0      0.3     18.9      for p in primes:
    26    118736      54985.0      0.5     32.8          while(n%p == 0):
    27     22389       8424.0      0.4      5.0              n = n/p
    28     22389       9536.0      0.4      5.7              factors.append(p)
    29     96347      41995.0      0.4     25.1          if(p > sqrt(n)):
    30      9999       3316.0      0.3      2.0              break
    31      9999       3809.0      0.4      2.3      if(n > 1):
    32      9596       4140.0      0.4      2.5          factors.append(n)

Looking at the line-by-line results, we observe that the line calling function ```fast_phi()``` requires most execution time inside ```main()```. However, in comparison with the implementation of ```phi()```, most time inside ```fast_phi()``` is now spent on factorizing the prime number inside ```factorize()```. We therefore see that the optimizations have had a significant impact. The new version avoids a double loop and significantly simplifies the logic.

Finally, we have a look at the memory consumption using ```memory_profiler```.

In [11]:
%mprun -T euler72.py.mprof -f euler72.main euler72.main()
print(open('euler72.py.mprof', 'r').read())

30397485.0


*** Profile printout saved to text file euler72.py.mprof. 
Filename: /home/paul/git/ASPP_course/day2_exercises/exc3_profiling/euler72.py

Line #    Mem usage    Increment   Line Contents
    62     52.2 MiB     52.2 MiB   def main(fast=True):
    63     52.2 MiB      0.0 MiB       primes = gen_primes(1000)
    64     52.2 MiB      0.0 MiB       m = 10000
    65                                 #m = 8
    66     52.2 MiB      0.0 MiB       fraq = 0
    67     52.2 MiB      0.0 MiB       if fast:
    68     52.2 MiB      0.0 MiB           for i in range(2,m+1):
    69     52.2 MiB      0.0 MiB               fraq += fast_phi(i,primes)
    70                                 else:
    71                                     for i in range(2,m+1):
    72                                         fraq += phi(i,primes)
    73                             
    74     52.2 MiB      0.0 MiB       print(fraq)
    75     52.2 MiB      0.0 MiB       return 0


The ```euler72``` program does not seem to consume measurable quantities of memory except for the intialization.

## 3c)

In order to optimize the code for speed, we replace many of the loops with generator expressions and make use of Python's built-in function ```sum()``` for Python arrays, which is expected to speed up the evaluation of the matrix multiplication, which was identified to be the bottleneck of this code in step 3a). This optimization is taken from:

https://www.programiz.com/python-programming/examples/multiply-matrix

In [12]:
%%capture time_mfo_main
%timeit mfo.main(print_result=True)

In [13]:
%%capture time_mf_main
%timeit mf.main(print_result=True)

In [14]:
print(str(time_mfo_main).split('\n')[-2])
%timeit mfo.main(print_result=False)

1.6 s +- 60.9 ms per loop (mean +- std. dev. of 7 runs, 1 loop each)
1.74 s ± 82.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [15]:
print(str(time_mf_main).split('\n')[-2])
%timeit mf.main(print_result=False)

3.83 s +- 209 ms per loop (mean +- std. dev. of 7 runs, 1 loop each)
4.01 s ± 134 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


From above's output we see that the execution time of our program has measurably improved due to the optimizations we have performed (it nearly halved). Using profilers, we investigate where the performance has been improved.

In [16]:
pff = %prun -r mfo.main(print_result=False)
pff.sort_stats('cumulative').print_stats()

          16473227 function calls in 3.572 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    3.823    3.823 {built-in method builtins.exec}
        1    0.001    0.001    3.823    3.823 <string>:1(<module>)
        1    0.000    0.000    3.822    3.822 matmult_functionalized_opt.py:23(main)
        1    0.000    0.000    3.536    3.536 matmult_functionalized_opt.py:7(matmult)
        1    0.003    0.003    3.536    3.536 matmult_functionalized_opt.py:8(<listcomp>)
    62750    1.515    0.000    3.320    0.000 {built-in method builtins.sum}
 15750250    1.805    0.000    1.805    0.000 matmult_functionalized_opt.py:8(<genexpr>)
   125250    0.041    0.000    0.248    0.000 random.py:218(randint)
   125250    0.090    0.000    0.206    0.000 random.py:174(randrange)
        1    0.000    0.000    0.153    0.153 matmult_functionalized_opt.py:17(get_Y)
        1    0.001    0.001    0.153    0.153

From the profiling results it seems that our improvements have decreased the time spent in functions ```matmult()```, ```get_X()```, and ```get_Y()``` by a little bit, but not by an amount that explains the difference in execution time observed earlier. Since profiling influences the execution speed and it might influence execution speeds of different implementations differntly, we cannot draw meaningful conclusions from here. Thus we move on to the ```line_profiler```. 

In [17]:
lprof_obj = %lprun -r -f mfo.matmult -f mfo.get_X -f mfo.get_Y -f mfo.main mfo.main(print_result=False)
lprof_obj.print_stats()

Timer unit: 1e-06 s

Total time: 5.90789 s
File: /home/paul/git/ASPP_course/day2_exercises/exc3_profiling/matmult_functionalized_opt.py
Function: matmult at line 7

Line #      Hits         Time  Per Hit   % Time  Line Contents
     7                                           def matmult(X,Y):
     8         1    5907891.0 5907891.0    100.0      return [[sum(a*b for a,b in zip(X_row,Y_col)) for Y_col in zip(*Y)] for X_row in X]

Total time: 0.318195 s
File: /home/paul/git/ASPP_course/day2_exercises/exc3_profiling/matmult_functionalized_opt.py
Function: get_X at line 11

Line #      Hits         Time  Per Hit   % Time  Line Contents
    11                                           def get_X(N):
    12                                               """Return random NxN numpy matrix
    13                                               """
    14         1     318195.0 318195.0    100.0      return [[random.randint(0,100) for j in range(N) ] for i in range(N)]

Total time: 0.306631 s
File:

We see that the relative time spent in the functions has changed significantly in comparison with the results from 3a). Now the matrix multiplication takes around 89% of the time, whereas it took around 98% of the time before. This means that we have managed to improve the matrix multiplication execution speed by more than we managed to decrease the array creation speed in ```get_X()``` and ```get_Y()```. Lastly we have a look at the memory consumption.

In [18]:
%mprun -T matmult_functionalized_opt.py.main.mprof -f mfo.main mfo.main(print_result=False)
print(open('matmult_functionalized_opt.py.mprof', 'r').read())



*** Profile printout saved to text file matmult_functionalized_opt.py.main.mprof. 
Filename: /home/paul/git/ASPP_course/day2_exercises/exc3_profiling/matmult_functionalized_opt.py

Line #    Mem usage    Increment   Line Contents
    27     51.6 MiB     51.6 MiB   def main(print_result=True):
    28                                 """Optimized main function binding together the other functions.
    29                                 """
    30                             #    random.seed(314156345)
    31     51.6 MiB      0.0 MiB       N = 250
    32     51.6 MiB      0.0 MiB       X = get_X(N)
    33     52.1 MiB      0.5 MiB       Y = get_Y(N)
    34     54.2 MiB      2.0 MiB       result = matmult(X,Y)
    35     54.2 MiB      0.0 MiB       if print_result:
    36                                     for r in result:
    37                                         print(r)
    38                                 
    39     54.2 MiB      0.0 MiB       return 0


The memory requirements of the code have not changed dramatically. However, I am under the impression that the ```memory_profiler``` is not the best quality software and that there are still quite a few rough edges in its implementation.